# Spotipy Features Dataframe

In [ ]:
pip install spotipy

In [7]:
import spotipy
import json
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="30da7dcb7eeb4f038924583a5dc6dfb3",
                                                           client_secret="d4d0d4cc03c3457bb24ec6dd80b17441"))


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import getpass
client_id = getpass.getpass(prompt='client_id: ', stream=None)
client_secret = getpass.getpass(prompt='client_secret: ', stream=None)

client_id: ········
client_secret: ········


In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

## Create a loop to search all 53 pages of the playlist results and add track info to a dictionary

In [4]:
def get_songs_artist(user, playlist_id):
    songs_artist = {}
    results = sp.user_playlist_tracks(user, playlist_id)
    for item in results['items']:
        song_name = item['track']['name']
        artist = item['track']['artists'][0]['name']
        song_id = item['track']['uri'].split(':')[-1]
        songs_artist[song_id] = {'song_name':song_name, 'artist':artist}
    while results['next']:
        results = sp.next(results)
        for item in results['items']:
            song_name = item['track']['name']
            artist = item['track']['artists'][0]['name']
            song_id = item['track']['uri'].split(':')[-1]
            songs_artist[song_id] = {'song_name':song_name, 'artist':artist}
    return songs_artist

In [24]:
songs = get_songs_artist("spotify", "4rnleEAOdmFAbRcNCgZMpY")

## Convert to DataFrame

In [25]:
data=pd.DataFrame(songs).T.reset_index()

In [26]:
data.head()

,index,song_name,artist
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier
1,2V4bv1fNWfTcyRJKmej6Sj,Cooler Than Me - Single Mix,Mike Posner
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator"
3,3gbBpTdY8lnQwqxNCcf795,Pompeii,Bastille
4,3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie (feat. Wyclef Jean),Shakira


## Using song ids in first column, add column for song features

In [9]:
combo_dataframe=data

In [10]:
combo_dataframe['audio features']=list(map(lambda x: sp.audio_features(x)[0],data['index']))

In [27]:
combo_dataframe.head()

,index,song_name,artist,audio features
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier,"{'danceability': 0.566, 'energy': 0.664, 'key'..."
1,2V4bv1fNWfTcyRJKmej6Sj,Cooler Than Me - Single Mix,Mike Posner,"{'danceability': 0.768, 'energy': 0.82, 'key':..."
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator","{'danceability': 0.558, 'energy': 0.559, 'key'..."
3,3gbBpTdY8lnQwqxNCcf795,Pompeii,Bastille,"{'danceability': 0.679, 'energy': 0.715, 'key'..."
4,3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie (feat. Wyclef Jean),Shakira,"{'danceability': 0.778, 'energy': 0.824, 'key'..."


## Create individual columns for the individual audio features 

In [12]:
audio_df=pd.json_normalize(combo_dataframe['audio features'])

In [28]:
audio_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.566,0.664,4,-5.303,0,0.0464,0.6340,0.000000,0.116,0.437,128.945,audio_features,7dS5EaCoMnN7DzlpT6aRn2,spotify:track:7dS5EaCoMnN7DzlpT6aRn2,https://api.spotify.com/v1/tracks/7dS5EaCoMnN7...,https://api.spotify.com/v1/audio-analysis/7dS5...,241688,4
1,0.768,0.820,7,-4.630,0,0.0474,0.1790,0.000000,0.689,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
2,0.558,0.559,6,-9.222,1,0.0959,0.3710,0.000007,0.109,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
3,0.679,0.715,9,-6.383,1,0.0407,0.0755,0.000000,0.271,0.571,127.435,audio_features,3gbBpTdY8lnQwqxNCcf795,spotify:track:3gbBpTdY8lnQwqxNCcf795,https://api.spotify.com/v1/tracks/3gbBpTdY8lnQ...,https://api.spotify.com/v1/audio-analysis/3gbB...,214148,4
4,0.778,0.824,10,-5.892,0,0.0707,0.2840,0.000000,0.405,0.758,100.024,audio_features,3ZFTkvIE7kyPt6Nu3PEa7V,spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V,https://api.spotify.com/v1/tracks/3ZFTkvIE7kyP...,https://api.spotify.com/v1/audio-analysis/3ZFT...,218093,4


## Concatenate both DataFrames

In [14]:
print(data.shape,audio_df.shape)

(5290, 4) (5290, 18)


In [15]:
thefinal = pd.concat([data, audio_df], axis=1, join='inner')

In [17]:
thefinal.shape

(5290, 22)

In [23]:
thefinal.head()

,index,song_name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,7dS5EaCoMnN7DzlpT6aRn2,Take Me To Church,Hozier,0.566,0.664,4,-5.303,0,0.0464,0.6340,...,0.116,0.437,128.945,audio_features,7dS5EaCoMnN7DzlpT6aRn2,spotify:track:7dS5EaCoMnN7DzlpT6aRn2,https://api.spotify.com/v1/tracks/7dS5EaCoMnN7...,https://api.spotify.com/v1/audio-analysis/7dS5...,241688,4
1,2V4bv1fNWfTcyRJKmej6Sj,Cooler Than Me - Single Mix,Mike Posner,0.768,0.820,7,-4.630,0,0.0474,0.1790,...,0.689,0.625,129.965,audio_features,2V4bv1fNWfTcyRJKmej6Sj,spotify:track:2V4bv1fNWfTcyRJKmej6Sj,https://api.spotify.com/v1/tracks/2V4bv1fNWfTc...,https://api.spotify.com/v1/audio-analysis/2V4b...,213293,4
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",0.558,0.559,6,-9.222,1,0.0959,0.3710,...,0.109,0.620,78.558,audio_features,7KA4W4McWYRpgf0fWsJZWB,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387,4
3,3gbBpTdY8lnQwqxNCcf795,Pompeii,Bastille,0.679,0.715,9,-6.383,1,0.0407,0.0755,...,0.271,0.571,127.435,audio_features,3gbBpTdY8lnQwqxNCcf795,spotify:track:3gbBpTdY8lnQwqxNCcf795,https://api.spotify.com/v1/tracks/3gbBpTdY8lnQ...,https://api.spotify.com/v1/audio-analysis/3gbB...,214148,4
4,3ZFTkvIE7kyPt6Nu3PEa7V,Hips Don't Lie (feat. Wyclef Jean),Shakira,0.778,0.824,10,-5.892,0,0.0707,0.2840,...,0.405,0.758,100.024,audio_features,3ZFTkvIE7kyPt6Nu3PEa7V,spotify:track:3ZFTkvIE7kyPt6Nu3PEa7V,https://api.spotify.com/v1/tracks/3ZFTkvIE7kyP...,https://api.spotify.com/v1/audio-analysis/3ZFT...,218093,4


In [ ]:
thefinal.to_csv('playlist_csv.csv')